In [1]:
import numpy as np
import pandas as pd
from src.dataloader_ import *
from src.network_ import *
from src.utils import *

import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd

train_series_dir = "../../inputs/series_train.parquet/"
test_series_dir = "../../inputs/series_test.parquet/"

data_dic_path = "../../inputs/data_dictionary.csv"
sample_submission_path = "../../inputs/sample_submission.csv"
train_path = "../../inputs/train.csv"
test_path = "../../inputs/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)
data_dic = pd.read_csv(data_dic_path)

In [2]:
train

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [3]:
def read_parquet(base_dir, id_):
    path = os.path.join(base_dir, f"id={id_}", "part-0.parquet")
    return pd.read_parquet(path)


def get_valid_ids(base_dir):
    return [f.split("=")[1].split(".")[0] for f in os.listdir(base_dir)]


p = read_parquet(base_dir="../../inputs/series_train.parquet/", id_="ffcd4dbd")

In [4]:
p

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,0.102994,0.553100,-0.251142,0.070094,-15.884976,0.0,5.0,4185.000000,56225000000000,2,1,48.0
1,1,-0.234882,0.870410,0.331544,0.028219,20.198034,0.0,2.0,4184.833496,56230000000000,2,1,48.0
2,2,0.578124,-0.319529,-0.167877,0.084935,-10.206018,0.0,0.0,4184.666504,62195000000000,2,1,48.0
3,3,-0.021039,0.264969,-0.526040,0.081154,-34.178265,0.0,2.0,4184.500000,62200000000000,2,1,48.0
4,4,0.760597,0.248146,0.223612,0.035003,13.265133,0.0,4.0,4184.333496,62205000000000,2,1,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
416270,416270,0.020685,0.006939,1.026014,0.026256,88.843529,0.0,1.5,3098.166748,69935000000000,5,1,72.0
416271,416271,0.020815,0.006631,1.025884,0.026126,88.869354,0.0,1.5,3098.166748,69940000000000,5,1,72.0
416272,416272,0.020789,0.007247,1.026131,0.026378,88.818832,0.0,1.5,3098.166748,69945000000000,5,1,72.0
416273,416273,0.020478,0.007119,1.026262,0.026501,88.850296,0.0,1.5,3098.166748,69950000000000,5,1,72.0


In [5]:
scale_columns = [
    "X",
    "Y",
    "Z",
    "enmo",
    "anglez",
    "light",
    "battery_voltage",
]
original_columns = ["non-wear_flag"]

p["masked_X"] = p["X"] * p["non-wear_flag"]
p["masked_Y"] = p["Y"] * p["non-wear_flag"]
p["masked_Z"] = p["Z"] * p["non-wear_flag"]
p["masked_enmo"] = p["enmo"] * p["non-wear_flag"]
p["masked_anglez"] = p["anglez"] * p["non-wear_flag"]
p["masked_light"] = p["light"] * p["non-wear_flag"]

# 0部分をnanにする
p["masked_X"] = p["masked_X"].replace(0, np.nan)
p["masked_Y"] = p["masked_Y"].replace(0, np.nan)
p["masked_Z"] = p["masked_Z"].replace(0, np.nan)
p["masked_enmo"] = p["masked_enmo"].replace(0, np.nan)
p["masked_anglez"] = p["masked_anglez"].replace(0, np.nan)
p["masked_light"] = p["masked_light"].replace(0, np.nan)


masked_columns = [
    "masked_X",
    "masked_Y",
    "masked_Z",
    "masked_enmo",
    "masked_anglez",
    "masked_light",
]


# 平均と分散を計算

# 列名に_meanをつける
mean_df = (
    p.groupby("relative_date_PCIAT")[scale_columns + masked_columns]
    .mean()
    .reset_index()
)
mean_df.columns = [
    f"{col}_mean" if col in scale_columns + masked_columns else col
    for col in mean_df.columns
]

# 列名に_stdをつける
std_df = (
    p.groupby("relative_date_PCIAT")[scale_columns + masked_columns].std().reset_index()
)
std_df.columns = [
    f"{col}_std" if col in scale_columns + masked_columns else col
    for col in std_df.columns
]

inputs = (
    pd.merge(mean_df, std_df, on="relative_date_PCIAT")
    .drop(columns=["relative_date_PCIAT"])
    .fillna(0.0)
)
inputs

,X_mean,Y_mean,Z_mean,enmo_mean,anglez_mean,light_mean,battery_voltage_mean,masked_X_mean,masked_Y_mean,masked_Z_mean,...,enmo_std,anglez_std,light_std,battery_voltage_std,masked_X_std,masked_Y_std,masked_Z_std,masked_enmo_std,masked_anglez_std,masked_light_std
0,0.338547,-0.150988,-0.359809,0.026116,-24.400423,5.392880,4176.479980,0.000000,0.000000,0.000000,...,0.064103,28.553017,14.649129,3.930875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.350519,-0.125009,-0.181064,0.023093,-13.276010,18.332214,4148.834961,0.000000,0.000000,0.000000,...,0.044709,34.584196,89.756758,12.016386,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.407191,-0.045867,-0.225568,0.045204,-15.191578,60.552517,4108.720703,0.000000,0.000000,0.000000,...,0.104193,34.590185,228.007505,11.367878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.332126,-0.141800,-0.338972,0.028174,-23.223186,21.023802,4073.753174,0.000000,0.000000,0.000000,...,0.070143,32.765924,77.712709,8.685680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.324830,-0.158355,-0.277500,0.032399,-18.782116,10.729764,4037.591309,0.000000,0.000000,0.000000,...,0.077859,31.799157,42.117143,13.529385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.386556,-0.266665,-0.279357,0.016017,-19.027714,7.699033,3997.822754,0.000000,0.000000,0.000000,...,0.033926,34.104112,10.675923,8.661207,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.465279,-0.151888,-0.272659,0.023765,-18.407175,38.180809,3971.875000,0.000000,0.000000,0.000000,...,0.058653,35.879790,215.555183,7.772528,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.366057,-0.107695,-0.298004,0.028913,-20.819143,17.627144,3945.766113,0.000000,0.000000,0.000000,...,0.070868,35.170675,92.733403,7.997540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.402301,-0.249367,-0.228722,0.012816,-14.992614,5.072811,3918.672119,0.000000,0.000000,0.000000,...,0.030061,29.000262,3.269554,7.704161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.381864,-0.065049,-0.250030,0.033165,-17.035446,11.246798,3887.920410,0.000000,0.000000,0.000000,...,0.077206,35.008554,47.828718,11.238922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
from glob import glob

# len(glob("../../normalized/*"))
len(glob("../../inputs/series_train.parquet/*"))

996

In [7]:
valid_ids = get_valid_ids(base_dir="../../normalized/")
dataset = LSTMAutoEncoderDataset(
    valid_ids=valid_ids, base_dir="../../inputs/series_train.parquet/"
)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=30)

model = LSTMAutoEncoder(input_size=26, hidden_size=13, num_layers=2).to("cuda")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model

LSTMAutoEncoder(
  (encoder): LSTM(26, 13, num_layers=2, batch_first=True)
  (decoder): LSTM(13, 26, num_layers=2, batch_first=True)
)

In [8]:
from tqdm import tqdm


for epoch in range(10):
    total_loss = 0
    for i, data in tqdm(enumerate(data_loader)):
        input_ = data["input"].to("cuda")
        target_ = data["output"].to("cuda")
        optimizer.zero_grad()
        output = model(input_)
        loss = criterion(output, target_)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"epoch: {epoch}, loss: {total_loss}")

0it [00:00, ?it/s]Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda-11.3/lib64/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
996it [00:20, 49.17it/s] 

epoch: 0, loss: 661.4673539926298



996it [00:20, 48.78it/s] 

epoch: 1, loss: 556.5711309798062



996it [00:20, 49.35it/s] 

...

epoch: 7, loss: 394.1541225463152



996it [00:20, 47.69it/s] 

epoch: 8, loss: 377.1229088976979



996it [00:20, 48.27it/s] 

epoch: 9, loss: 363.55346086621284
